In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

In [83]:
import pandas as pd
df1 = pd.read_csv('./data/pred_val_bert.csv')
df1.columns = ['review_id', 'bert1']
df1['bert2'] = pd.read_csv('./data/pred_val_bert_2.csv')['stars']
df1['bert3'] = pd.read_csv('./data/pred_val_bert_3.csv')['stars']
df1['rnn'] = pd.read_csv('./data/valid_pred.csv')['stars']
df1['lgb'] = pd.read_csv('./data/lgb_val.csv')['predicted_star'].round()
df1.head()

,review_id,bert1,bert2,bert3,rnn,lgb
0,ewaMkuxfjfCX3ndtk0oRTw,3,3,3,5,4.0
1,SzkuvIpf0lcVaDgk5x2ipg,5,5,5,5,4.0
2,ZGg6emL3sqhyyqHD_DJEag,5,5,4,3,3.0
3,IXYLmBVvRVFJZEhnhE_YsQ,5,5,5,5,3.0
4,dao7SEXhBuMk7TieM4gEZA,3,3,2,3,3.0


In [3]:
df1['cat'] = 'valid'

In [4]:
df2 = pd.read_csv('./data/lgb_pred.csv')
df2.columns = ['review_id', 'lgb']
df2['bert1'] = pd.read_csv('./data/pred_bert.csv')['stars']
df2['bert2'] = pd.read_csv('./data/pred_bert_2.csv')['stars']
df2['bert3'] = pd.read_csv('./data/pred_bert_3.csv')['stars']
df2['rnn'] = pd.read_csv('./data/test_pred.csv')['stars']

df2.head()

,review_id,lgb,bert1,bert2,bert3,rnn
0,SE3NWk7exO4LhLrG-4fq8A,3.743851,3,4,4,3
1,HDjbIqJJNqgOLezeikEA_w,4.721213,4,5,4,2
2,_TrAAObnxO9ZHfY6twWB1w,4.268382,5,5,5,5
3,jqjtghMH3Uy6mGYk7k9Heg,3.259250,5,5,5,1
4,0c4Tn5kRB09dByvmcEWxbw,4.147467,5,5,5,5


In [104]:
(df1.bert3 == df1.rnn).value_counts()

True     1436
False     564
dtype: int64

In [105]:
(df2.bert3 == df2.rnn).value_counts()

False    1311
True      689
dtype: int64

In [106]:
(df1.bert3 == df1.lgb.round()).value_counts()

False    1040
True      960
dtype: int64

In [107]:
(df2.bert3 == df2.lgb.round()).value_counts()

False    1055
True      945
dtype: int64

In [5]:
pd.read_csv('./data/test.csv')

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,3D-FJxp8bsBzm3zJVqs92g,0,2016-07-26,0,SE3NWk7exO4LhLrG-4fq8A,0.0,Got the breakfast sandwich with home fries bot...,0,4wf350SdjZs3PQeYTjoilA
1,OIdOJaNS8M624F58XGV3PQ,1,2012-10-15,0,HDjbIqJJNqgOLezeikEA_w,0.0,@pizzalibretto\nI booked a large group here fo...,3,0BBUmH7Krcax1RZgbH4fSA
2,Cq-quzfGpVgcBPpZ6WzCKw,0,2017-01-07,0,_TrAAObnxO9ZHfY6twWB1w,0.0,AMAZING!! I got a reasonable quote and they ca...,0,wHw29VJKpu1IXJ0tXznaXQ
3,BDV4499NWVKQgQQ5FjPd8w,6,2011-01-05,9,jqjtghMH3Uy6mGYk7k9Heg,0.0,Disclaimer: \nI'm not a tea expert. I don't ow...,8,OgSkk8AjZpDfxJQxOZlT-w
4,z1F2e1xHWMe6FxYvTReWcw,0,2017-06-26,0,0c4Tn5kRB09dByvmcEWxbw,0.0,I have been going here for years and expect a ...,0,nZihvJzDqLSTd0E0Q_VFvg
...,...,...,...,...,...,...,...,...,...
1995,wPiRP76arN0NB3M1ZXXp7g,0,2009-08-25,0,b7nE8KfrTQOU8QiZiLvlDA,0.0,Great for large groups! I was looking for a re...,0,8ebQm-Py_UxW73Sc0SlPLg
1996,1NmGVWYlF4iMngM6arKJTQ,1,2013-12-30,0,TsEzGvNX8Q0aJXGZBfYQ2w,0.0,3.5 stars\nThis place has become our neighborh...,2,ZvmC522FAwxt8qT1jxqliw
1997,gEeEpUAXI36N57nvCYJMxg,0,2014-07-28,3,yuwbV4vtYheZBW4gMDLxgQ,0.0,So just had my first experience here and I was...,2,EzjLARgz3Jmb4L1t3lBy1A
1998,Q2Gd5Dmqtii59EheSjsA-Q,0,2018-06-28,0,uJSHhBzlC8GzZgAm_YN6TQ,0.0,"Ok, this place HAS to be officially closed. It...",0,31LqBA0cb06FodcApSyNeQ


In [6]:
df2['cat'] = 'test'

In [7]:
df = pd.concat([df1, df2])
df['cat'].value_counts()

test     2000
valid    2000
Name: cat, dtype: int64

In [8]:
def round_star(star):
    if star < 1:
        return 1
    elif star > 5:
        return 5
    else:
        return round(star)

In [9]:
df.columns

Index(['bert1', 'bert2', 'bert3', 'cat', 'lgb', 'review_id', 'rnn'], dtype='object')

In [10]:
import numpy as np
df['all_mean'] = df.apply(
    lambda x: np.array([x['bert1'], x['bert2'], x['bert3'], x['rnn'], x['lgb']]).mean(), axis=1)
df['all_mean'] = df['all_mean'].apply(lambda x: round_star(x))
df.head()

,bert1,bert2,bert3,cat,lgb,review_id,rnn,all_mean
0,3,3,3,valid,4.360081,ewaMkuxfjfCX3ndtk0oRTw,5,4
1,5,5,5,valid,3.824301,SzkuvIpf0lcVaDgk5x2ipg,5,5
2,5,5,4,valid,2.929254,ZGg6emL3sqhyyqHD_DJEag,3,4
3,5,5,5,valid,3.088829,IXYLmBVvRVFJZEhnhE_YsQ,5,5
4,3,3,2,valid,2.831013,dao7SEXhBuMk7TieM4gEZA,3,3


In [11]:
va = pd.read_csv('./data/valid.csv')
y_val = va['stars']

In [12]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def eva(method):
    y_val_pre = df[df.cat == 'valid'][method]
    acc = accuracy_score(y_val, y_val_pre)
    p, r, f1, _ = precision_recall_fscore_support(y_val, y_val_pre, average="macro")
    print("accuracy:", acc, "\tprecision:", p, "\trecall:", r, "\tf1:", f1)

In [13]:
eva('all_mean')

accuracy: 0.685 	precision: 0.6247420830079342 	recall: 0.6122394922290675 	f1: 0.6171661965616136


In [14]:
def normalized_mean(lst):
    lst.sort()
    lst = lst[1:-1]
    return np.array(lst).mean()

df['norm_mean'] = df.apply(
    lambda x: normalized_mean([x['bert1'], x['bert2'], x['bert3'], x['rnn'], x['lgb']]), axis=1)
df['norm_mean'] = df['all_mean'].apply(lambda x: round_star(x))
df.head()

,bert1,bert2,bert3,cat,lgb,review_id,rnn,all_mean,norm_mean
0,3,3,3,valid,4.360081,ewaMkuxfjfCX3ndtk0oRTw,5,4,4
1,5,5,5,valid,3.824301,SzkuvIpf0lcVaDgk5x2ipg,5,5,5
2,5,5,4,valid,2.929254,ZGg6emL3sqhyyqHD_DJEag,3,4,4
3,5,5,5,valid,3.088829,IXYLmBVvRVFJZEhnhE_YsQ,5,5,5
4,3,3,2,valid,2.831013,dao7SEXhBuMk7TieM4gEZA,3,3,3


In [15]:
eva('norm_mean')

accuracy: 0.685 	precision: 0.6247420830079342 	recall: 0.6122394922290675 	f1: 0.6171661965616136


In [16]:
df['lgb_round'] = df['lgb'].apply(lambda x: round_star(x))
eva('bert1'), eva('bert2'), eva('bert3'), eva('rnn'), eva('lgb_round')

accuracy: 0.6955 	precision: 0.6212697377840808 	recall: 0.5987076558751883 	f1: 0.6027102765173292
accuracy: 0.684 	precision: 0.6162061359268345 	recall: 0.6003210947296718 	f1: 0.6062267975508856
accuracy: 0.6995 	precision: 0.6253411371368436 	recall: 0.6173560306045883 	f1: 0.6186924975842738
accuracy: 0.6475 	precision: 0.5452207599623294 	recall: 0.5311110738929457 	f1: 0.5226389273140538
accuracy: 0.4485 	precision: 0.4687899120153495 	recall: 0.40073731519981315 	f1: 0.39445434093244286


(None, None, None, None, None)

In [17]:
df.shape

(4000, 10)

In [18]:
df['majority_5'] = df.apply(
    lambda x: np.bincount(np.array(
        [x['bert1'], x['bert2'], x['bert3'], x['rnn'], x['lgb_round']])).argmax(), axis=1)
df['majority_5'] = df['majority_5'].apply(lambda x: round_star(x))
df.head()

,bert1,bert2,bert3,cat,lgb,review_id,rnn,all_mean,norm_mean,lgb_round,majority_5
0,3,3,3,valid,4.360081,ewaMkuxfjfCX3ndtk0oRTw,5,4,4,4,3
1,5,5,5,valid,3.824301,SzkuvIpf0lcVaDgk5x2ipg,5,5,5,4,5
2,5,5,4,valid,2.929254,ZGg6emL3sqhyyqHD_DJEag,3,4,4,3,3
3,5,5,5,valid,3.088829,IXYLmBVvRVFJZEhnhE_YsQ,5,5,5,3,5
4,3,3,2,valid,2.831013,dao7SEXhBuMk7TieM4gEZA,3,3,3,3,3


In [19]:
eva('majority_5')

accuracy: 0.706 	precision: 0.6297323774772653 	recall: 0.6158952470483465 	f1: 0.6194785128621503


In [20]:
df.columns

Index(['bert1', 'bert2', 'bert3', 'cat', 'lgb', 'review_id', 'rnn', 'all_mean',
       'norm_mean', 'lgb_round', 'majority_5'],
      dtype='object')

In [21]:
import numpy as np
df['bert_mean'] = df.apply(
    lambda x: np.array([x['bert1'], x['bert2'], x['bert3']]).mean(), axis=1)
df['bert_mean'] = df['bert_mean'].apply(lambda x: round_star(x))
df.head()

eva('bert_mean')

accuracy: 0.7035 	precision: 0.6325868610858352 	recall: 0.6200954529231575 	f1: 0.6256466337097656


In [22]:
df

,bert1,bert2,bert3,cat,lgb,review_id,rnn,all_mean,norm_mean,lgb_round,majority_5,bert_mean
0,3,3,3,valid,4.360081,ewaMkuxfjfCX3ndtk0oRTw,5,4,4,4,3,3
1,5,5,5,valid,3.824301,SzkuvIpf0lcVaDgk5x2ipg,5,5,5,4,5,5
2,5,5,4,valid,2.929254,ZGg6emL3sqhyyqHD_DJEag,3,4,4,3,3,5
3,5,5,5,valid,3.088829,IXYLmBVvRVFJZEhnhE_YsQ,5,5,5,3,5,5
4,3,3,2,valid,2.831013,dao7SEXhBuMk7TieM4gEZA,3,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,4,4,5,test,4.648547,b7nE8KfrTQOU8QiZiLvlDA,5,5,5,5,5,4
1996,4,4,4,test,4.218852,TsEzGvNX8Q0aJXGZBfYQ2w,1,3,3,4,4,4
1997,4,4,4,test,2.354180,yuwbV4vtYheZBW4gMDLxgQ,3,3,3,2,4,4
1998,1,1,1,test,2.948256,uJSHhBzlC8GzZgAm_YN6TQ,1,1,1,3,1,1


In [23]:
val = df[df.cat == 'valid'][['review_id', 'bert_mean']]
val.columns = ['review_id', 'stars']
val.to_csv('./data/val_bert_mean.csv', index=False)
tst = df[df.cat == 'test'][['review_id', 'bert_mean']]
tst.columns = ['review_id', 'stars']
tst.to_csv('./data/tst_bert_mean.csv', index=False)

In [24]:
acc = accuracy_score(y_val, val['stars'])
p, r, f1, _ = precision_recall_fscore_support(y_val, val['stars'], average="macro")
print("accuracy:", acc, "\tprecision:", p, "\trecall:", r, "\tf1:", f1)

accuracy: 0.7035 	precision: 0.6325868610858352 	recall: 0.6200954529231575 	f1: 0.6256466337097656


In [42]:
tst.to_csv('./data/pred.csv', index=False)

In [98]:
test_check = pd.read_csv('./data/pred_bert_3.csv')
test_check['rnn'] = pd.read_csv('./data/lgb_pred.csv')['predicted_star'].round()
(test_check['rnn'] == test_check.stars).value_counts()

False    1055
True      945
dtype: int64

In [100]:
test_check = pd.read_csv('./data/pred_bert_3.csv')
test_check['rnn'] = pd.read_csv('./data/pred_val_bert_3.csv')['stars']
(test_check['rnn'] == test_check.stars).value_counts()

False    1385
True      615
dtype: int64

In [47]:
import lightgbm as lgb
from tqdm import tqdm
from sklearn.model_selection import ParameterGrid

X = df[df.cat == 'valid'][['bert1', 'bert2', 'bert3', 'rnn', 'lgb']]
y = y_val

train = lgb.Dataset(
    data=X, 
    label=y)

# tune the parameter with grid search (with cross validation)
param_grid = {
    'max_depth': [3, 5, 7, None],
    'min_data_in_leaf': [1, 4, 8, 16]
}

cv_results = []

for hyperparams in tqdm(list(ParameterGrid(param_grid))):
    fixed = {
        "objective": 'regression',
        "learning_rate": 0.01
    }
    hyperparams.update(fixed)

    validation_summary = lgb.cv(
        hyperparams,
        train,
        num_boost_round=6000,
        nfold=5,
        metrics=["rmse"],
        early_stopping_rounds=500,
        verbose_eval=None)

    optimal_num_trees = len(validation_summary["rmse-mean"])
    hyperparams["num_boost_round"] = optimal_num_trees
    cv_results.append((hyperparams, validation_summary["rmse-mean"][-1]))
    
tmp = pd.DataFrame(cv_results)
param = list(tmp.sort_values(by=[1])[0])[0]
train = lgb.Dataset(data=X[:1000], label=y[:1000])
gbm = lgb.train(
    param, 
    train_set=train, 
    num_boost_round=param['num_boost_round'])

100%|██████████| 16/16 [00:48<00:00,  3.61s/it]


In [49]:
acc = accuracy_score(y[1000:], [round_star(s) for s in gbm.predict(X[1000:])])
p, r, f1, _ = precision_recall_fscore_support(y_val, y_val_pre, average="macro")
print("accuracy:", acc, "\tprecision:", p, "\trecall:", r, "\tf1:", f1)

accuracy: 0.675 	precision: 0.6158562692864693 	recall: 0.5986992496273191 	f1: 0.6054109712017525
